In [127]:
import import_ipynb
from library import *
from function_And_Class import get_image_paths, Imagetransform, load_model, Model
from bien_URL import data_directory_Test, data_directory_animals, save_path_resnet

In [128]:
use_pretrain = True
model = models.resnet50(pretrained = use_pretrain)
model.fc = nn.Linear(model.fc.in_features, 120) 
model_load = load_model(save_path_resnet, model)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/pbang/Downloads/model_resnet.pth'

In [ ]:
# Lưu ảnh và nhãn vào hai list
data_animal_1, labels_1 = get_image_paths(data_directory_Test)
data_animal_2, labels_2 = get_image_paths(data_directory_animals)
data_animal = data_animal_1 + data_animal_2
labels = list(set(labels_1 + labels_2))
labels = sorted(labels)

In [ ]:
# Loại bỏ các nhãn trùng lặp (ví dụ: cattle với cow thì loại bỏ cattle)
labels.remove("ladybugs")
labels.remove('moth')
labels.remove('cattle')

In [ ]:
# Tạo hai dict để đối chiếu từ nhãn sang số và số sang nhãn
label_to_id = {name: idx for idx, name in enumerate(labels)}
id_to_label = {idx: name for name, idx in label_to_id.items()}

In [ ]:
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        drop(file_path)


def process_image(image_path):
    """ Xử lý ảnh và dự đoán """
    image = Image.open(image_path).convert("RGB")
    image = image.resize((224, 224))
    img_tk = ImageTk.PhotoImage(image)
    image_label.config(image=img_tk)
    image_label.image = img_tk

    # Chuyển đổi ảnh thành tensor
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    image_tensor = transform(image).unsqueeze(0)

    # Dự đoán
    prediction = model.predict(image_tensor)
    result_label.config(text=f"Dự đoán: {id_to_label[prediction]}")
    result_label.place(relx=0.5, rely=0.55, anchor="center")

def reset():
    """Khôi phục trạng thái ban đầu"""
    image_label.place_forget()  # Ẩn ảnh dự đoán
    label.place(relx=0.5, rely=0.3, anchor="center")  # Hiện lại vùng kéo thả
    result_label.place_forget() # Xóa nhãn dự đoán
    btn_reset.place_forget()  # Ẩn nút Reset
    upload_btn.place(relx=0.5, rely=0.8, anchor="center")
    drag_leave(True)


def drop(event):
    """ Xử lý khi kéo thả ảnh vào """
    if isinstance(event, str):
        file_path = event.strip("{}")
    else:
        file_path = event.data.strip("{}")  # Xóa dấu {} nếu có
    label.place_forget()
    image_label.place(relx=0.5, rely=0.3, anchor="center")
    process_image(file_path)
    # Hiển thị nút reset
    btn_reset.place(relx=0.5, rely=0.8, anchor="center")
    upload_btn.place_forget()


# Hiệu ứng đổi màu viền khi kéo ảnh vào
def drag_enter(event):
    label.config(bg="lightgray", bd=4, relief="ridge")  # Viền nổi lên


def drag_leave(event):
    label.config(bg="gray", bd=2, relief="solid")  # Viền trở lại bình thường

In [ ]:
from tkinter import Label, Button, filedialog


In [ ]:

# Khởi tạo mô hình
model = Model(model_load)

# Khởi tạo cửa sổ Tkinter hỗ trợ kéo thả
root = TkinterDnD.Tk()
root.title("Phân loại ảnh động vật")
root.geometry("600x600")

# Mở ảnh nền
bg_image = Image.open("C:/Users/pbang/Downloads/hinh-nen-nen-xanh-duong-powerpoint-cute_092909120.jpg")  # Đưa ảnh nền vào
bg_image = bg_image.resize((600, 600), Image.Resampling.LANCZOS)  # Resize cho vừa cửa sổ
bg_photo = ImageTk.PhotoImage(bg_image)

# Tạo label để chứa ảnh nền
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(relwidth=1, relheight=1)  # Phủ toàn bộ cửa sổ

# Vùng kéo thả ảnh
label = Label(root, text="Đưa ảnh vào đây", bg="gray", fg="white", width=50, height=10)
label.place(relx=0.5, rely=0.3, anchor="center")  # Đặt giữa màn hình

# Hiển thị ảnh
image_label = Label(root)

# Hiển thị kết quả
result_label = Label(root, text="", font=("Arial", 14, "bold"))

upload_btn = tk.Button(root, text="Chọn ảnh", command=upload_image, font=("Arial", 12), bg="#FFD700", fg="black", relief="raised")
upload_btn.place(relx=0.5, rely=0.8, anchor="center")

btn_reset = tk.Button(root, text="Reset", command=reset, font=("Arial", 12), bg="#FF4500", fg="white", relief="raised", padx=10, pady=5)

# Kết nối sự kiện kéo thả
label.drop_target_register(DND_FILES)
label.dnd_bind("<<Drop>>", drop)
label.dnd_bind("<<DropEnter>>", drag_enter)
label.dnd_bind("<<DropLeave>>", drag_leave)
# Chạy chương trình
root.mainloop()
